In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

%cd ../..

!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin05.jureca


In [2]:
import h5py as h5
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
def load_volume(clustering, method):
    path = f"data/aa/volume/{clustering}/{method}/{clustering}.h5"
    with h5.File(path, 'r') as f:
        volume = f['volume'][:]
    return path, volume

def iou_cluster(volume, cluster_ix, ignore_ix=0):
    vol_rolled = np.roll(volume, 1, axis=1)
    vol_clip = volume[:, 1:, :]
    vol_rolled_clip = vol_rolled[:, 1:, :]

    hits1 = vol_rolled_clip[vol_clip != ignore_ix] == cluster_ix
    hits2 = vol_clip[vol_clip != ignore_ix] == cluster_ix

    intersection = np.sum(hits1 & hits2)
    union = np.sum(hits1 | hits2)

    return intersection / union

In [4]:
methods = [
    "simclr-imagenet",
    "pli_glcm_testset",
    "pli_sobel_histo_testset",
    "resnet50_planes8_962-1083_same",
    "resnet50_planes8_962-1083_circle_small",
    "resnet50_planes8_962-1083_circle",
    "resnet50_planes8_962-1083_neighbor",
    "resnet50_planes8_962-1083_sphere_small",
    "resnet50_planes8_962-1083_sphere",
]
clusterings = [
    "kmeans_pca80_s1_2",
    "kmeans_pca80_s1_8",
    "kmeans_pca80_s1_32",
    "kmeans_pca80_s1_128"
]

###

iou_frame = pd.DataFrame()

for m in methods:
    for c in clusterings:
        print(m, c)

        path, volume = load_volume(c, m)
        print(path)

        cluster_ix = np.unique(volume)[1:]

        ious = []
        for c_ix in cluster_ix:
            ious.append(iou_cluster(volume, c_ix))

        mean_iou = np.mean(ious)

        print("Mean IoU:", mean_iou)

        iou_frame = iou_frame.append({'method': m, 'clustering': c, 'iou_list': ious, 'mean_iou': mean_iou}, ignore_index=True)

iou_frame = iou_frame.sort_values(['clustering', 'method']).reset_index(drop=True)

simclr-imagenet kmeans_pca80_s1_2
data/aa/volume/kmeans_pca80_s1_2/simclr-imagenet/kmeans_pca80_s1_2.h5
Mean IoU: 0.7778976568467562
simclr-imagenet kmeans_pca80_s1_8
data/aa/volume/kmeans_pca80_s1_8/simclr-imagenet/kmeans_pca80_s1_8.h5
Mean IoU: 0.50857232589854
simclr-imagenet kmeans_pca80_s1_32
data/aa/volume/kmeans_pca80_s1_32/simclr-imagenet/kmeans_pca80_s1_32.h5
Mean IoU: 0.35453011687859026
simclr-imagenet kmeans_pca80_s1_128
data/aa/volume/kmeans_pca80_s1_128/simclr-imagenet/kmeans_pca80_s1_128.h5
Mean IoU: 0.23603364124934972
pli_glcm_testset kmeans_pca80_s1_2
data/aa/volume/kmeans_pca80_s1_2/pli_glcm_testset/kmeans_pca80_s1_2.h5
Mean IoU: 0.95206710086184
pli_glcm_testset kmeans_pca80_s1_8
data/aa/volume/kmeans_pca80_s1_8/pli_glcm_testset/kmeans_pca80_s1_8.h5
Mean IoU: 0.5836165850580097
pli_glcm_testset kmeans_pca80_s1_32
data/aa/volume/kmeans_pca80_s1_32/pli_glcm_testset/kmeans_pca80_s1_32.h5
Mean IoU: 0.3406761008738666
pli_glcm_testset kmeans_pca80_s1_128
data/aa/volume/k

In [6]:
models = ['resnet50_planes8_962-1083', 'pli_sobel_histo_testset', 'pli_glcm_testset', 'simclr-imagenet']

def method_split(method):
    model = None
    for m in models:
        if method.startswith(m):
            model = m
    context = method.split(model)[-1]
    if len(context) > 0 and context[0] == "_":
        context = context[1:]
    return model, context

for k, r in iou_frame.iterrows():
    m, c = method_split(r['method'])
    iou_frame.loc[k, 'method'] = m
    iou_frame.loc[k, 'context'] = c

iou_frame['mean_iou'] = (iou_frame['mean_iou'] * 100).round(1)

In [13]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_2'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
17,simclr-imagenet,kmeans_pca80_s1_2,"[0.7660259479009119, 0.7897693657926004]",77.8,
14,resnet50_planes8_962-1083,kmeans_pca80_s1_2,"[0.8350358703358709, 0.9435332513957013]",88.9,same
13,resnet50_planes8_962-1083,kmeans_pca80_s1_2,"[0.9001173888832513, 0.8845951998014139]",89.2,neighbor
10,pli_sobel_histo_testset,kmeans_pca80_s1_2,"[0.9306851821989602, 0.9600836303143557]",94.5,
11,resnet50_planes8_962-1083,kmeans_pca80_s1_2,"[0.9589500408012501, 0.9404581143210295]",95.0,circle
9,pli_glcm_testset,kmeans_pca80_s1_2,"[0.9647864176789923, 0.9393477840446877]",95.2,
12,resnet50_planes8_962-1083,kmeans_pca80_s1_2,"[0.9643153871898037, 0.9428308304992445]",95.4,circle_small
15,resnet50_planes8_962-1083,kmeans_pca80_s1_2,"[0.9597926277866867, 0.9486480198772252]",95.4,sphere
16,resnet50_planes8_962-1083,kmeans_pca80_s1_2,"[0.9660442529726626, 0.9551486935899013]",96.1,sphere_small


In [14]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_8'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
32,resnet50_planes8_962-1083,kmeans_pca80_s1_8,"[0.46580449518373174, 0.19992656021683664, 0.7...",47.1,same
35,simclr-imagenet,kmeans_pca80_s1_8,"[0.5223338275083247, 0.5255210808749572, 0.426...",50.9,
29,resnet50_planes8_962-1083,kmeans_pca80_s1_8,"[0.7257229091831253, 0.33210916891033126, 0.72...",55.8,circle
27,pli_glcm_testset,kmeans_pca80_s1_8,"[0.5336318013587366, 0.4231894251748139, 0.740...",58.4,
28,pli_sobel_histo_testset,kmeans_pca80_s1_8,"[0.35105170832479987, 0.698100068631882, 0.557...",58.7,
30,resnet50_planes8_962-1083,kmeans_pca80_s1_8,"[0.2649320276591088, 0.7295072955972716, 0.777...",61.8,circle_small
34,resnet50_planes8_962-1083,kmeans_pca80_s1_8,"[0.6380081497675448, 0.6496570508157009, 0.797...",70.5,sphere_small
31,resnet50_planes8_962-1083,kmeans_pca80_s1_8,"[0.8850511703612497, 0.5599430241058254, 0.906...",70.8,neighbor
33,resnet50_planes8_962-1083,kmeans_pca80_s1_8,"[0.5364008986142763, 0.8223263192455602, 0.780...",71.9,sphere


In [15]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_32'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
23,resnet50_planes8_962-1083,kmeans_pca80_s1_32,"[0.4220616097631793, 0.08318153271238286, 0.33...",25.0,same
19,pli_sobel_histo_testset,kmeans_pca80_s1_32,"[0.4153748741760561, 0.1315352658307972, 0.303...",32.4,
18,pli_glcm_testset,kmeans_pca80_s1_32,"[0.3780331304811458, 0.1677288888845344, 0.549...",34.1,
26,simclr-imagenet,kmeans_pca80_s1_32,"[0.4293615076069251, 0.35246097703088103, 0.22...",35.5,
21,resnet50_planes8_962-1083,kmeans_pca80_s1_32,"[0.49818978535565395, 0.4287028063999367, 0.16...",35.9,circle_small
20,resnet50_planes8_962-1083,kmeans_pca80_s1_32,"[0.2439326420069245, 0.511141320743333, 0.2439...",36.7,circle
22,resnet50_planes8_962-1083,kmeans_pca80_s1_32,"[0.7000522979597494, 0.42771774082624225, 0.59...",50.2,neighbor
24,resnet50_planes8_962-1083,kmeans_pca80_s1_32,"[0.26873519076170965, 0.6767854593900176, 0.47...",50.7,sphere
25,resnet50_planes8_962-1083,kmeans_pca80_s1_32,"[0.39340527070377274, 0.6282877888584255, 0.46...",51.0,sphere_small


In [16]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_128'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
5,resnet50_planes8_962-1083,kmeans_pca80_s1_128,"[0.05283786539599799, 0.33777463302312194, 0.0...",12.8,same
1,pli_sobel_histo_testset,kmeans_pca80_s1_128,"[0.08396772576013128, 0.25204744881377966, 0.1...",18.6,
0,pli_glcm_testset,kmeans_pca80_s1_128,"[0.18166920285613952, 0.11360544217687076, 0.2...",19.4,
3,resnet50_planes8_962-1083,kmeans_pca80_s1_128,"[0.12084320673637833, 0.530917945690673, 0.092...",19.9,circle_small
2,resnet50_planes8_962-1083,kmeans_pca80_s1_128,"[0.1977038245157507, 0.09951302331737462, 0.25...",21.0,circle
8,simclr-imagenet,kmeans_pca80_s1_128,"[0.18020624376278943, 0.1445358469336793, 0.17...",23.6,
4,resnet50_planes8_962-1083,kmeans_pca80_s1_128,"[0.1737346705630629, 0.17924555306833992, 0.54...",30.3,neighbor
6,resnet50_planes8_962-1083,kmeans_pca80_s1_128,"[0.26055270943069486, 0.21338691902383394, 0.2...",32.0,sphere
7,resnet50_planes8_962-1083,kmeans_pca80_s1_128,"[0.3936514126787016, 0.22880164857601398, 0.30...",32.3,sphere_small
